### Libraries to use

In [20]:
import os
import warnings
from pyspark import SparkContext
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder, StringIndexer, MinMaxScaler
from pyspark.ml import Pipeline

warnings.filterwarnings('ignore')
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages  org.apache.spark:spark-avro_2.12:3.4.1,io.delta:delta-core_2.12:2.4.0 pyspark-shell'

In [ ]:
spark = SparkSession.builder.master("local[*]").appName("Spark_ML") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
        .enableHiveSupport()\
        .getOrCreate()

sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

### Data reading

In [10]:
df = spark.read.format('delta')\
    .load('delta/saber11')

Number of rows and schema

In [11]:
df.count()

23/07/05 00:26:15 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


10361

In [12]:
df.printSchema()

root
 |-- lenguaje: string (nullable = true)
 |-- matematicas: string (nullable = true)
 |-- sociales: string (nullable = true)
 |-- Id: integer (nullable = true)
 |-- filosofia: string (nullable = true)
 |-- biologia: string (nullable = true)
 |-- quimica: string (nullable = true)
 |-- fisica: string (nullable = true)
 |-- nivel_ingles: string (nullable = true)
 |-- sisben: integer (nullable = true)
 |-- estrato: integer (nullable = true)
 |-- genero: string (nullable = true)
 |-- puntaje_saber11: string (nullable = true)



In [13]:
for column in df.columns:
    if column != 'Id':
        print(f'\nUnique values at column {column}\n')
        df.select(column).distinct().show()


Unique values at column lenguaje



+--------+
|lenguaje|
+--------+
|    bajo|
|   medio|
|superior|
|    alto|
+--------+


Unique values at column matematicas

+-----------+
|matematicas|
+-----------+
|       bajo|
|      medio|
|   superior|
|       alto|
+-----------+


Unique values at column sociales

+--------+
|sociales|
+--------+
|    bajo|
|   medio|
|superior|
|    alto|
+--------+


Unique values at column filosofia

+---------+
|filosofia|
+---------+
|     bajo|
|    medio|
| superior|
|     alto|
+---------+


Unique values at column biologia

+--------+
|biologia|
+--------+
|    bajo|
|   medio|
|superior|
|    alto|
+--------+


Unique values at column quimica

+--------+
| quimica|
+--------+
|    bajo|
|   medio|
|superior|
|    alto|
+--------+


Unique values at column fisica

+--------+
|  fisica|
+--------+
|    bajo|
|superior|
|   medio|
|    alto|
+--------+


Unique values at column nivel_ingles

+------------+
|nivel_ingles|
+------------+
|          A2|
|          A-|
|          B1|
|    

### Feature transform

In [14]:
df = df.drop('id')
df = df.dropna()

Target to binary values

In [19]:
df = df.withColumn('puntaje_saber11',
                   when(col('puntaje_saber11')=='bajo', 0)\
                   .otherwise(1))

#### String Indexer, OneHotEncoder and MinMaxScaler

In [51]:
# features for OHE and MinMaxScaler

OHE_cols = [x[0] for x in df.dtypes if x[1] == 'string' and x[0] != 'puntaje_saber11']
to_scale = [x[0] for x in df.dtypes if x[1] in ['int', 'float'] and x[0] != 'puntaje_saber11']

In [97]:
# Multiple Indexers for every column in OHE_cols
indexer = [StringIndexer(inputCol = col, outputCol = col+'_indexed') for col in OHE_cols]

# Multiple OHE for every column in OHE_cols
encoder = [OneHotEncoder(inputCol = col+'_indexed', outputCol = col+'_encoded') for col in OHE_cols]

aux_features = [col+'_encoded' for col in OHE_cols] + to_scale

# Vector Assembler to can apply MinMaxScaler
vector_to_scale = [VectorAssembler(inputCols = [col], outputCol = col+'_vec') for col in aux_features]
scaler = [MinMaxScaler(inputCol = col+'_vec', outputCol = col+'_scaled') for col in aux_features]

# Final Vector Assembler
vector = VectorAssembler(inputCols = [col+'_scaled' for col in aux_features], outputCol ='features')

transform_steps = indexer + encoder + vector_to_scale + scaler

# Pipeline
features_pipeline = Pipeline(stages = transform_steps)
df_trans = features_pipeline.fit(df).transform(df)
to_select = [col for col in df_trans.columns if '_scaled' in col or col == 'puntaje_saber11']
df_trans.select(to_select).show(5)

+---------------+-----------------------+--------------------------+-----------------------+------------------------+-----------------------+----------------------+---------------------+---------------------------+---------------------+--------------------+--------------------+
|puntaje_saber11|lenguaje_encoded_scaled|matematicas_encoded_scaled|sociales_encoded_scaled|filosofia_encoded_scaled|biologia_encoded_scaled|quimica_encoded_scaled|fisica_encoded_scaled|nivel_ingles_encoded_scaled|genero_encoded_scaled|       sisben_scaled|      estrato_scaled|
+---------------+-----------------------+--------------------------+-----------------------+------------------------+-----------------------+----------------------+---------------------+---------------------------+---------------------+--------------------+--------------------+
|              0|          [1.0,0.0,0.0]|             [1.0,0.0,0.0]|          [1.0,0.0,0.0]|           [1.0,0.0,0.0]|          [0.0,1.0,0.0]|         [0.0,1.0,0.0]

### Modeling

Baseline (logistic regression)

In [99]:
from pyspark.ml.classification import LogisticRegression

# to_vector = to_select
# to_vector.append('puntaje_saber11')
model_lr = LogisticRegression(featuresCol = 'features', labelCol = 'puntaje_saber11')
pipeline_lr = Pipeline(stages = [features_pipeline, vector, model_lr])

In [100]:
(trainingData, testingData) = df.randomSplit([0.70, 0.3])
lr_model = pipeline_lr.fit(trainingData)

In [101]:
lr_summary = lr_model.stages[2].summary.predictions
lr_summary = lr_summary.select('features', 'probability' ,'prediction')
lr_summary.show(10, truncate = False)

+---------------------------------------------------------------------------+----------------------------------------+----------+
|features                                                                   |probability                             |prediction|
+---------------------------------------------------------------------------+----------------------------------------+----------+
|(28,[0,3,6,10,12,15,18,22,25],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])       |[0.25516373223123756,0.7448362677687624]|1.0       |
|(28,[0,3,6,10,12,15,18,22,25],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])       |[0.25516373223123756,0.7448362677687624]|1.0       |
|(28,[0,3,6,10,12,15,18,22,25],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])       |[0.25516373223123756,0.7448362677687624]|1.0       |
|(28,[0,3,6,10,12,15,18,22,25],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])       |[0.25516373223123756,0.7448362677687624]|1.0       |
|(28,[0,3,6,10,12,15,18,22,25],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])       |[0.2551637322

In [102]:
test = lr_model.transform(testingData).withColumnRenamed("puntaje_saber11", "label")

In [103]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol = 'prediction', labelCol = 'label')
evaluator.evaluate(test,  {evaluator.metricName: "accuracy"})

0.6779062299293513